In [1]:
print("ok")

ok


In [1]:
%pwd

'/Users/saideepthimakineni/Documents/AI/Nutrition-Assistant-Chatbot/research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'/Users/saideepthimakineni/Documents/AI/Nutrition-Assistant-Chatbot'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [6]:
extracted_data=load_pdf_file(data='Data/')

In [7]:
extracted_data

[Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-02-12T05:28:21+00:00', 'source': 'Data/Nutritions.pdf', 'total_pages': 191, 'page': 0, 'page_label': '1'}, page_content='ix\nNutrition is one of the most important factors that impact health in all\nareas of the lifecycle. Pregnant women need adequate food and health care\nto deliver a healthy baby who has a good birth weight and a fighting chance\nfor survival. In many regions of the world, the infant mortality rate is very\nhigh, meaning that many infants will not live to see their first birthday.\nBreastfeeding is the ideal method of feeding and nurturing infants, because\nbreast milk contains many immunologic agents that protect the infant against\nbacteria, viruses, and parasites. Yet, less than 40 percent of infants world-\nwide are exclusively breastfed (no other food or drink, not even water) for\nthe first four months of life. Children need adequate nutrition to develop\nand gr

In [8]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("length of all the Text chunks", len(text_chunks))

length of all the Text chunks 1381


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [11]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

/var/folders/zr/d8t0ctj501d_2x862l96cjn00000gn/T/ipykernel_5532/4043855499.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/Users/saideepthimakineni/anaconda3/envs/bot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
query_result = embeddings.embed_query("Hello world")
print("length", len(query_result))

length 384


In [18]:
#query_result

In [14]:
!rm -rf ~/anaconda3/envs/bot/lib/python3.10/site-packages/pinecone*

zsh:1: no matches found: /Users/saideepthimakineni/anaconda3/envs/bot/lib/python3.10/site-packages/pinecone*


In [15]:

!python -m pip uninstall -y pinecone pinecone-client pinecone-plugin-inference pinecone-plugin-interface


In [16]:
!python -m pip install --no-deps --upgrade pinecone-client

  Using cached pinecone_client-5.0.1-py3-none-any.whl.metadata (19 kB)
Using cached pinecone_client-5.0.1-py3-none-any.whl (244 kB)


In [17]:
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [18]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [19]:
!python -m pip uninstall pinecone-client pinecone pinecone-plugin-inference pinecone-plugin-interface -y


Found existing installation: pinecone-client 5.0.1
Uninstalling pinecone-client-5.0.1:
  Successfully uninstalled pinecone-client-5.0.1


In [20]:
!python -m pip install --upgrade pinecone-client


  Using cached pinecone_client-5.0.1-py3-none-any.whl.metadata (19 kB)
  Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl.metadata (1.2 kB)
Using cached pinecone_client-5.0.1-py3-none-any.whl (244 kB)
Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl (85 kB)
Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl (6.2 kB)


In [21]:
!python -m pip list | grep pinecone

langchain-pinecone        0.2.0
pinecone-client           5.0.1
pinecone-plugin-inference 1.1.0
pinecone-plugin-interface 0.0.7


In [22]:
import os
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "samplebot"
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
 )

In [23]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [24]:
from langchain_pinecone import PineconeVectorStore
docsearch =PineconeVectorStore.from_documents(
  documents=text_chunks,
  index_name=index_name,
  embedding=embeddings,
)

In [25]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
  index_name=index_name,
  embedding=embeddings
)

In [26]:
docsearch

In [27]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [28]:
retrieved_docs = retriever.invoke("what is nutrition ?")

In [29]:
retrieved_docs

[Document(id='f6de9d2c-d377-426b-a610-5dbaa43bfc48', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2025-02-12T05:28:21+00:00', 'page': 20.0, 'page_label': '21', 'producer': 'iLovePDF', 'source': 'Data/Nutritions.pdf', 'total_pages': 191.0}, page_content='Adult Nutrition\nThe science of nutrition is dedicated to learning about foods that the hu-\nman body requires at different stages of life in order to meet the nutritional\nneeds for proper growth, as well as to maintain health and prevent disease.\nA baby is born with a very high requirement for energy and nutrient in-\ntake per unit of body weight to provide for rapid growth. The rate of growth\nis the highest during the first year and declines slowly after the age of two,'),
 Document(id='eb505928-eb9d-49ba-8072-177227d18e01', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2025-02-12T05:28:21+00:00', 'page': 143.0, 'page_label': '144', 'producer': 'iLovePDF', 'source': 'Data/Nutritions.pdf', 'total_pag

In [30]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [31]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an AI assistant designed to answer questions accurately and concisely. "
    "Use the provided information to generate responses. "
    "If the answer is not available in the provided context, respond with 'I am sorry, I don't know. Thanks for understanding !'"
    "Keep your response within three sentences and avoid unnecessary details.\n\n"
    "Context:\n{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)


In [32]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [33]:
response = rag_chain.invoke({"input": "What is Nutrition?"})
print(response["answer"])



System: Nutrition is the maintenance of health through proper eating, or the study of the same. It involves understanding the types of foods eaten and the dietary substances necessary for health.


In [36]:
response = rag_chain.invoke({"input": "What is Makeup?"})
print(response["answer"])


I am sorry, I don't know. Thanks for understanding!
